In [13]:
import networkx as nx
import matplotlib.pyplot as plt
import math
import statistics

data_path = './data/'
reduced_graph_path = data_path + 'wiki-topcats-reduced.txt'
page_names_path = data_path + 'wiki-topcats-page-names.txt'
categories_path = data_path + 'wiki-topcats-categories.txt'

In [14]:
# Bulding the graph with networkx
H = nx.read_edgelist(path=reduced_graph_path, delimiter="\t", create_using=nx.DiGraph())

### RQ1

In [3]:
print(len(H.edges.items()))
print(len(H.nodes.items()))
print(nx.is_directed(H))

'''
In mathematics, a dense graph is a graph in which the number
of edges is close to the maximal number of edges.
'''
print(nx.density(H))

# mean degree value
mean = []

for n in H.nodes.items():
    mean.append(nx.degree(H, n[0]))

print(sum(mean) / len(mean))

2645247
461193
True
1.2436602635647606e-05
11.471323285479182


-We can treat the wikipedia hyperliks graph as a subset of the webgraph, that is of course directed, since is possible (and common) that a page has a link to another and not vice versa, so the edge has to be directed.

-We used networkx library's functions to find the number of nodes and edges:
- Nodes are **N=2645247**
- Edges are **L=461193**

-Average node degree is equal to **11.47** so every wikipedia page has , on average, 11.5 hyperlinks from and to other wikipedia articles, since degree is the sum of in-degree and out-degree

-In order to say if wikipedia hyperlinks graph is dense or not we hate to consider the number of edges compared to the number of all possible edges that is $N(N-1)$ so density of our graph is:

$$ \Delta = \frac{L}{N(N-1)} = \frac{461193}{6997329045762} = 0.000012 $$
We can conclude that our graph is not dense, it is actually very sparse.

### RQ2

We need to preprocess wiki-topcats-categories.txt file as we should consider categories that have 3500 articles, at least. We could do this using the famous bash tool "awk":

```bash
awk 'NF > 3501' data/wiki-topcats-categories.txt > data/wiki-topcats-categories__3500.txt
```

- NF is defined as 'Number of Fields' indicating the number of items inside a row (more generally, columns)
- The value of 3501 is justified as we are considering the Category label (ex. Category:Telugu_actors; 581455 581966 582010 582033 582071 ... )

Eventually, we count the lines of the resulting file (using another terminal tool called "wc")

```bash
wc -l data/wiki-topcats-categories__3500.txt
```

and we notice that the set of Categories has been narrowed down to **35** items from **17364**.

```bash
cut -f 1 -d ' ' data/wiki-topcats-categories__3500.txt
```

- "f" is the field position (or first columns, in this case)
- "d" is the delimiter (space)

These are the categories that satisfy our threshold

```
Category:English_footballers;
Category:The_Football_League_players;
Category:Association_football_forwards;
Category:Association_football_goalkeepers;
Category:Association_football_midfielders;
Category:Association_football_defenders;
Category:Living_people;
Category:Year_of_birth_unknown;
Category:Harvard_University_alumni;
Category:Major_League_Baseball_pitchers;
Category:Members_of_the_United_Kingdom_Parliament_for_English_constituencies;
Category:Indian_films;
Category:Year_of_death_missing;
Category:English_cricketers;
Category:Year_of_birth_missing_(living_people);
Category:Rivers_of_Romania;
Category:Main_Belt_asteroids;
Category:Asteroids_named_for_people;
Category:English-language_albums;
Category:English_television_actors;
Category:British_films;
Category:English-language_films;
Category:American_films;
Category:Fellows_of_the_Royal_Society;
Category:People_from_New_York_City;
Category:American_Jews;
Category:American_television_actors;
Category:American_film_actors;
Category:Debut_albums;
Category:Black-and-white_films;
Category:Year_of_birth_missing;
Category:Place_of_birth_missing_(living_people);
Category:Article_Feedback_Pilot;
Category:American_military_personnel_of_World_War_II;
Category:Windows_games;
```

In [4]:
# This has to be a list of lists where each element represents a category (list of articles)
# The highest is the value, the furthest a category is
block_ranking = []

# indexes
categ2articles = {}
articles2categ = {}

# category used as C_0 (minimum set of items)
min_categ_len = math.inf
min_categ = (None, None)

# with open(data_path + 'wiki-topcats-categories__3500.txt', 'r') as most_linked_categs:
with open(categories_path, 'r') as most_linked_categs:
    for i, line in enumerate(most_linked_categs):
        line = line.rstrip('\n')
        line = line.split()
        
        # get name and articles for each category
        categ_name, articles, articles_len = line[0].lstrip('Category:').rstrip(';'), line[1:], len(line[1:])
        
        # save the smaller categ within a certain length
        if articles_len < min_categ_len and articles_len >= 350:
            min_categ_len = articles_len
            min_categ = (categ_name, articles)
        
        # discard the ones under 3500 items...
        if articles_len <= 3500: continue
        
        # categ2articles
        if not categ_name in categ2articles:
            categ2articles[categ_name] = set(articles)
        else:
            categ2articles[categ_name].update(articles)
        
        # articles2categ
        for a in articles:
            if not a in articles2categ.items():
                articles2categ[a] = set([categ_name])
            else:
                articles2categ[a].add(categ_name)

# putting C_0 in place
# we chose a small category to speed up code runtime
block_ranking.insert(0, min_categ[0])

In [5]:
print('Categories:', len(categ2articles.items()))
print('Smallest category:', min_categ[0], len(min_categ[1]))
print('Category in position 0:', list(categ2articles.items())[0][0], len(list(categ2articles.items())[0][1]))
print('Category in position 1:', list(categ2articles.items())[1][0], len(list(categ2articles.items())[1][1]))

Categories: 35
Smallest category: UEFA_Euro_2000_players 350
Category in position 0: English_footballers 9237
Category in position 1: The_Football_League_players 9467


Now, we should compute the Shortest Path algorithm to get the nearest and the furthest Category from C0. To be fast enough to show some results we'll use networkx library shortest path algorithm, even though we wrote our dijkstra function (that's more a modified BFS as we're dealing with a not-weighted graph).

In [6]:
# whole set of categories (except the min_categ)
categ_list = list(categ2articles.items())

# min_categ (used as C_0) taken into account to speed up the runtime
c_zero = min_categ

# shortest path for each category
distances = [(min_categ[0], 0)] * len(categ_list)

# save up some time storing and retrieving
# already computed paths
distances_memoization = {}

# stats
skipped_memoization = 0
skipped_no_path = 0
skipped_no_nodes_in_H = 0
total = 0

# shortest path
def compute_sp(c0, ci):
    
    # use global variables for stats
    global total, skipped_memoization, skipped_no_path, skipped_no_nodes_in_H
    
    # distance values
    sp_values = []
    
    # for each node inside the input category C_0
    for c_x in c0:
        
        # for each node inside the i-st category C_i
        for c_y in ci:
            total += 1
            
            # key used for memoization
            key = c_x + '_' + c_y
            
            # if we've already computed it, use it!
            if key in distances_memoization:
                
                # save value for later use
                sp_values.append(distances_memoization[key])
                skipped_memoization += 1
                continue
            
            # skip in case of neither c_x nor c_y are contained inside the graph
            if not c_x in H or not c_y in H:
                skipped_no_nodes_in_H += 1
                continue
            
            # if there's not any path, set infinite as value 
            if not nx.has_path(H, c_x, c_y):
                sp_values.append(math.inf)
                
                # save for later use
                distances_memoization[key] = math.inf
                skipped_no_path += 1
                continue
            
            # we defined our own shortest function but to be fast enough
            # we're using the one from the library
            shortest_path = nx.shortest_path(H, c_x, c_y)
            shortest_path_len = len(shortest_path)
            
            # append the path length value
            sp_values.append(shortest_path_len)
            
            # save for later use
            distances_memoization[key] = shortest_path_len
    
    # return the median if the length is sufficient
    return statistics.median(sorted(sp_values)) if len(sp_values) > 0 else sp_values

# Filling and sorting our block_ranking
for i in range(1, len(categ_list)):
    
    # i-st category 
    c_ist = categ_list[i]
    
    # info
    print('processing', c_zero[0], 'and', c_ist[0])
    
    # computing distances using a batch of nodes to be fast enough
    # remove the slicing to work on the full set of nodes related to the c-st category
    distances[i] = (c_ist[0], compute_sp(list(c_zero[1]), list(c_ist[1])[:50]))

# show our block ranking computed on a subset of data
print(sorted(distances, key=lambda x: x[1]))

processing UEFA_Euro_2000_players and The_Football_League_players
processing UEFA_Euro_2000_players and Association_football_forwards
processing UEFA_Euro_2000_players and Association_football_goalkeepers
processing UEFA_Euro_2000_players and Association_football_midfielders
processing UEFA_Euro_2000_players and Association_football_defenders
processing UEFA_Euro_2000_players and Living_people
processing UEFA_Euro_2000_players and Year_of_birth_unknown
processing UEFA_Euro_2000_players and Harvard_University_alumni
processing UEFA_Euro_2000_players and Major_League_Baseball_pitchers
processing UEFA_Euro_2000_players and Members_of_the_United_Kingdom_Parliament_for_English_constituencies
processing UEFA_Euro_2000_players and Indian_films
processing UEFA_Euro_2000_players and Year_of_death_missing
processing UEFA_Euro_2000_players and English_cricketers
processing UEFA_Euro_2000_players and Year_of_birth_missing_(living_people)
processing UEFA_Euro_2000_players and Rivers_of_Romania
proc

In [7]:
# info about computed path, skipped, retrieved from a dict to be re-used (memoization) 
print(total, skipped_memoization, skipped_no_path, skipped_no_nodes_in_H)

595000 51340 157598 90100


In [8]:
# We add our min_categ (350 items) to categ2articles since it has been excluded from
# the creation of this data structure.
# This category is needed in order to compute the score.
categ2articles[min_categ[0]] = min_categ[1]
print(categ2articles[min_categ[0]])

['27108', '27301', '27305', '27377', '27383', '27402', '27450', '27459', '27561', '27562', '27566', '27607', '27994', '28057', '73449', '73491', '73642', '73643', '73664', '73666', '73810', '73811', '74001', '74083', '76809', '76810', '76811', '76906', '76910', '78971', '78978', '80393', '80395', '80396', '80397', '80709', '81320', '81321', '81463', '81609', '81826', '81827', '81828', '81829', '81854', '81858', '81867', '81885', '81915', '81926', '81942', '82051', '82058', '82071', '82091', '82317', '82321', '82366', '82368', '82369', '82393', '82476', '82477', '82537', '82735', '82738', '82740', '82742', '82745', '82970', '83047', '83472', '83500', '83507', '83512', '83513', '83514', '83516', '83526', '83527', '83528', '83954', '83958', '83960', '83984', '83985', '83988', '84176', '84178', '84279', '84328', '84384', '84385', '84391', '84392', '84393', '84845', '84848', '84850', '84879', '84892', '84893', '84957', '84958', '84959', '84966', '85008', '85214', '85221', '85414', '85422', 

In [9]:
# we compute the score using only 50 nodes
# to be faster but less accurate. We decided to proceed using a subset.
# Running the code without slicing will take hours but it'll work.
score = {n: 0 for n in H.nodes()} 
visited, sub_H_nodes = set(), list()

# loop over category
for cat in categ2articles.keys():
    
    # taking a batch of nodes to be fast enough
    # remove the slicing to work on the full set of nodes 
    cat_nodes = list(categ2articles[cat])[:50]
    
    # info about the processed categories and their batches of nodes
    print(cat, len(cat_nodes))
    
    # induced subgraph through networkx
    sub_H_nodes.extend(cat_nodes)
    I = H.subgraph(sub_H_nodes)
    
    for n in cat_nodes:
        
        # skip node (related to the category) if not contained inside the induced subgraph
        if not n in I: continue
        
        # for each predecessor (the nodes that lie behind of n)
        for predecessor in list(I.predecessors(n)):
            
            # compute the score
            score[n] += score[predecessor] if predecessor in visited else 1
    
    # set all nodes in this category as visited
    for n in cat_nodes:
        visited.add(n)

English_footballers 50
The_Football_League_players 50
Association_football_forwards 50
Association_football_goalkeepers 50
Association_football_midfielders 50
Association_football_defenders 50
Living_people 50
Year_of_birth_unknown 50
Harvard_University_alumni 50
Major_League_Baseball_pitchers 50
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 50
Indian_films 50
Year_of_death_missing 50
English_cricketers 50
Year_of_birth_missing_(living_people) 50
Rivers_of_Romania 50
Main_Belt_asteroids 50
Asteroids_named_for_people 50
English-language_albums 50
English_television_actors 50
British_films 50
English-language_films 50
American_films 50
Fellows_of_the_Royal_Society 50
People_from_New_York_City 50
American_Jews 50
American_television_actors 50
American_film_actors 50
Debut_albums 50
Black-and-white_films 50
Year_of_birth_missing 50
Place_of_birth_missing_(living_people) 50
Article_Feedback_Pilot 50
American_military_personnel_of_World_War_II 50
Windows_games 50
UEFA_E

In [10]:
# page-names
# this code help to debug and to show results (basically, it links a node id to pagename)
page2names = {}
names2page = {}

# open page-name file
with open(page_names_path, 'r') as page_names:
    for line in page_names:
        node_id, node_name = line.rstrip('\n').split(' ', 1)
        
        # page2names
        page2names[node_id] = node_name
        
        # names2page
        names2page[node_name] = node_id

In [11]:
# info
print(page2names['52'])
print(names2page['Hung Huang'])

Hung Huang
52


What we did in step 1, 2 and 3 was to create a induced subgraph from each category (sequentially) and to score their nodes. If the predecessor of the node came from the same category, the assigned score is one. Otherwise, if it came from precedent categories, they heir their score. The process ends when a massive graph has been create after processing all categories, eventually, obtaining the same graph as the originalmo 

In [28]:
score_sorted = sorted(score.items(), key=lambda x: x[1], reverse=True)

print('---', 'FIRST 50' ,'---')

for n_k, n_v in score_sorted[:50]:
    print(page2names[n_k])

print('\n---', 'LAST 50' ,'---')

for n_k, n_v in score_sorted[-50:]:
    print(page2names[n_k])

--- FIRST 50 ---
Steve McManaman
Robbie Fowler
Tony Adams (footballer)
Steven Gerrard
Zinedine Zidane
Nick Barmby
Henning Berg
Roar Strand
Paul Ince
Vegard Heggem
PlayStation 3
Progressivism
Gheorghe Hagi
Bogdan Stelea
Dorinel Munteanu
Burt Lancaster
Richard Avedon
Warcraft III: Reign of Chaos
Nigel Martyn
Dan Eggen
Martin Keown
Peter Schmeichel
Richard Wright (footballer)
Gareth Barry
Sol Campbell
The Noah's Ark Trap
From the Devil to a Stranger
Fabio Coltorti
Cyril Washbrook
Graham Atkinson (cricketer)
Frank Rost
Hung Huang
Anna Wintour
Chen Kaige
Oprah Winfrey
Richard Cytowic
Oliver Sacks
Dmitri Nabokov
United States
David Eagleman
Jay Hambidge
Ronald Reagan
James Brady
American University
Tim Tebow
Cretien van Campen
Philosophy
James Wannerton
CBS
Ty Segall

--- LAST 50 ---
Sonny Caldinez
Lesley Scott
Keith Temple
Mat Irvine
Robbie Stamp
Tim Child
Rosala Mera
Amancio Ortega Gaona
Pablo Isla
Lester Estelle II
Morning's Wrath
John L. Hines, Jr.
This Will Destroy You (album)
Khalid Ab